In [1]:
# Setting up Imports 

import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Setting up Model Thresholds

model_memory_threshold = 60 # in KB

In [3]:
# Setting up Model Directories

models_dir = '../models/session1'
models = os.listdir(models_dir)
models

['2.keras', '0.keras', '1.keras']

In [24]:
# Get data

def get_data(input_shape):
    sample_nums = 1
    dims = [sample_nums]
    dims = dims + list(input_shape[:-1])
    data = tf.random.normal(shape = dims)
    data = (np.expand_dims(data, axis=-1))
    return data

In [25]:
# Model querying

for i in models:
    model = tf.keras.models.load_model(os.path.join(models_dir, i))
    input_dims = (model.layers[0].input_shape[0])[1:]
    data = get_data(input_dims)
    total = 0
    for i in range(1000):
        data = get_data(input_dims)
        tf.config.experimental.reset_memory_stats('GPU:0')
        model.predict(data, verbose=None)
        results = tf.config.experimental.get_memory_info('GPU:0')
        results = (results["peak"] - results["current"])/1000
        total = results + total
    results = total / 1000
    print(f"Memory used:",results,"KB")

Memory used: 157524.04070400292 KB
Memory used: 135841.83347199878 KB
Memory used: 135858.62681600108 KB


In [19]:
tf.config.experimental.reset_memory_stats('GPU:0')
tf.config.experimental.get_memory_info('GPU:0')

{'current': 249048320, 'peak': 249048320}

In [26]:
converter = tf.lite.TFLiteConverter.from_keras_model(os.path.join(models_dir, models[0]))
tflite_model = converter.convert()

    # Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

AttributeError: 'str' object has no attribute 'call'